In [1]:
import torch

In [2]:
inputs=torch.tensor(
    [
     [0.43,0.15,0.89],#your
     [0.55,0.87,0.66],#journey
     [0.57,0.85,0.64],#starts
     [0.22,0.58,0.33],#with
     [0.77,0.25,0.10],#one
     [0.05,0.80,0.55],#step
    ]
)

# Convert input embedding to Key,Query and Value

In [3]:
x_2=inputs[1]
d_in=inputs.shape[1]
d_out=2

In [4]:
#initialize three weight matrix wq,wk,wv
torch.manual_seed(123)
w_query=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_key=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
w_value=torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)

print(w_query)

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])


In [5]:
#query,key,value for second inputs journey, matrix multiplications
x2_query=x_2@w_query
x2_key=x_2@w_key
x2_value=x_2@w_value

print(x2_query)

tensor([0.4306, 1.4551])


In [6]:
query=inputs@w_query
key=inputs@w_key
value=inputs@w_value

print(query.shape)

torch.Size([6, 2])


# Compute attention score

In [7]:
keys_2=key[1]
attn_score_22=query[1].dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [8]:
#now lets compute the attention score for all
attn_score_2=query[1]@key.T#all attention score for journey

print(attn_score_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [9]:
attn_scores=query@key.T
print(attn_scores)

tensor([[0.9231, 1.3545, 1.3241, 0.7910, 0.4032, 1.1330],
        [1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440],
        [1.2544, 1.8284, 1.7877, 1.0654, 0.5508, 1.5238],
        [0.6973, 1.0167, 0.9941, 0.5925, 0.3061, 0.8475],
        [0.6114, 0.8819, 0.8626, 0.5121, 0.2707, 0.7307],
        [0.8995, 1.3165, 1.2871, 0.7682, 0.3937, 1.0996]])


# now lets Normalize the attention score

In [10]:
d_k=key.shape[1]
print(d_k)
attn_weight=torch.softmax(attn_scores/d_k**0.5, dim=1)
print(attn_weight)

2
tensor([[0.1551, 0.2104, 0.2059, 0.1413, 0.1074, 0.1799],
        [0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820],
        [0.1503, 0.2256, 0.2192, 0.1315, 0.0914, 0.1819],
        [0.1591, 0.1994, 0.1962, 0.1477, 0.1206, 0.1769],
        [0.1610, 0.1949, 0.1923, 0.1501, 0.1265, 0.1752],
        [0.1557, 0.2092, 0.2048, 0.1419, 0.1089, 0.1794]])


# Context vector Embedding

In [15]:
j_attn=attn_weight[1]
print(j_attn)
print(value)
print(j_attn@value)


tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])
tensor([[0.1855, 0.8812],
        [0.3951, 1.0037],
        [0.3879, 0.9831],
        [0.2393, 0.5493],
        [0.1492, 0.3346],
        [0.3221, 0.7863]])
tensor([0.3061, 0.8210])


In [22]:
all_context=attn_weight@value
print(all_context)


tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]])


In [29]:
class Self_Attention(torch.nn.Module):
    def __init__(self,d_in,d_out):
        super().__init__()
        self.w_query=torch.nn.Parameter(torch.rand(d_in,d_out))
        self.w_key=torch.nn.Parameter(torch.rand(d_in,d_out))
        self.w_value=torch.nn.Parameter(torch.rand(d_in,d_out))

    def forward(self,input_embedding):
        query=input_embedding@self.w_query
        key=input_embedding@self.w_key
        value=input_embedding@self.w_value

        attn_scores=query@key.T
        attn_weight=torch.softmax(attn_scores/key.shape[1]**0.5,dim=1)

        context_vector=attn_weight@value

        return context_vector
    
torch.manual_seed(123)
attn=Self_Attention(d_in,d_out)
context_vec=attn.forward(inputs)
print(context_vec)


tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [35]:
#use nn.Linear in stead nn.Parameter to initialize weight because it initialize  more optimize weit matrix 


class Self_Attention(torch.nn.Module):
    def __init__(self,d_in,d_out,qkv_bias=False):
        super().__init__()
        self.w_query=torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_key=torch.nn.Linear(d_in,d_out,bias=qkv_bias)
        self.w_value=torch.nn.Linear(d_in,d_out,bias=qkv_bias)
    def forward(self,input_embedding):
        query=self.w_query(input_embedding)
        key=self.w_key(input_embedding)
        value=self.w_value(input_embedding)

        attn_scores=query@key.T
        attn_weight=torch.softmax(attn_scores/key.shape[1]**0.5,dim=1)

        context_vector=attn_weight@value

        return context_vector
    
torch.manual_seed(123)
attn=Self_Attention(d_in,d_out)
context_vec=attn.forward(inputs)
print(context_vec)


tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)
